# Script that sets up training repository

Change the settings in the first cell and run all to create the repository and send the invitations.

In [1]:
import os, sys, json,importlib, base64,urllib

#This notebook requires you to install https://github.com/PyGithub/PyGithub first
from github import Github, InputGitAuthor

#Add parent folder to library paths
module_path = os.path.abspath(os.path.join('../../Common-Resources'))
if module_path not in sys.path:
    sys.path.append(module_path)
import github_tools as ghTools

In [2]:
accessToken = ghTools.getTokenAttribute(attribute="token")
ghSession = Github(accessToken)

#get user
user = ghSession.get_user()
username = user.login

### Manually input name of repository

In [3]:
#The name of the repository to create, use random 4 letters to get unique url
reponame = "lyrics-Jul14"

## Creating the repo

In [4]:
#making sure the repo is not already tere
try:
    del_repo = ghSession.get_repo(username+"/"+reponame)
    print("Deleted: " + del_repo.full_name)
    del_repo.delete()
except:
    print("No repo with that name")

Deleted: kbjarkefur/lyrics-Jul14


In [5]:
#Create the repo
training_repo = user.create_repo(reponame, auto_init=True)
print(training_repo)

Repository(full_name="kbjarkefur/lyrics-Jul14")


### Replace the auto initialized repo with the prepared README and add .gitignore

#### update README

In [6]:
#Get the commit sha from where the README was created in the auto_init
contents = training_repo.get_contents("README.md")

#Read the README file in this repo to get the content for the commit
file_content = open('repo-content-files/README.md', 'rb').read()
#print(file_content)

#Update the README file
readme_update_result = training_repo.update_file("/README.md", 
                                                 "Update README.md", 
                                                 file_content,
                                                 contents.sha,
                                                 branch="master")

readme_update_sha = readme_update_result["commit"].sha
print(readme_update_result)

{'commit': Commit(sha="ff6d627ad530f7cd14e56e636e1f9cd2d1382981"), 'content': ContentFile(path="README.md")}


#### Create the initial-branch

In [7]:
training_repo.create_git_ref("refs/heads/initial-branch", readme_update_sha)

GitRef(ref="refs/heads/initial-branch")

#### add .gitignore

In [8]:
#Read the README file in this repo to get the content for the commit
file_content = open("repo-content-files/no.gitignore", 'rb').read()

#Update the README file
create_ignore_result = training_repo.create_file("/.gitignore", "add a ignore file", file_content, branch="master")

### Start adding lyrics files

In [9]:
list_of_files = [
    ["/lyrics/pop/backstreets-back.txt","Adding Backstreet Back - Go BSB!"],
    ["/lyrics/rock/stairway-to-heaven.txt","Stairway to Heaven! So GOOOD."],
    ["/lyrics/rock/november-rain.txt","November Rain - The solo is AMAZING!"],
    ["/lyrics/pop/mamma-mia.txt","ABBA is the best! And this song is their best!"]
]

for song_list in list_of_files:
    #Read the README file in this repo to get the content for the commit
    file_content = open("repo-content-files/" + song_list[0], 'rb').read()
    #print(file_content)

    #Update the README file
    create_result = training_repo.create_file(song_list[0], song_list[1], file_content, branch="master")
    
last_sha = create_result["commit"].sha
print(last_sha)

bb6aa37a21d525f53266e9ba87bdff8c061a81ca


In [10]:
#Create the queen branch
training_repo.create_git_ref("refs/heads/queen-branch", last_sha)

GitRef(ref="refs/heads/queen-branch")

In [11]:
#Remove Abba from the queen branch
contents = training_repo.get_contents("lyrics/pop/mamma-mia.txt", ref="refs/heads/queen-branch")
training_repo.delete_file("/"+contents.path, "ABBA is not that great", contents.sha, branch="queen-branch")

{'commit': Commit(sha="cea824bebd7236ee7af2218379e11d021e21cf69"),
 'content': NotSet}

In [12]:
list_of_files = [
    ["/lyrics/pop/dont-stop-me-now.txt","Dont try to stop me!, Yass Queen","queen-branch"]
]

for song_list in list_of_files:
    #Read the README file in this repo to get the content for the commit
    file_content = open("repo-content-files/" + song_list[0], 'rb').read()
    #print(file_content)

    #Update the README file
    create_result = training_repo.create_file(song_list[0], song_list[1], file_content, branch=song_list[2])
    
    last_sha = create_result["commit"].sha
    print(create_result)

{'content': ContentFile(path="lyrics/pop/dont-stop-me-now.txt"), 'commit': Commit(sha="6a740e20c1e636edd3b5a741c01d70e189ca1be3")}
